## 시계열분석을 통한 가스 소비량 예측

### Index
### 1. 데이터 수집 및 탐색
### 2. 분석을 위한 데이터 전처리
### 3. 머신러닝 모델 구조 생성
### 4. 머신러닝 훈련셋 학습
### 5. 머신러닝 시험셋 예측
### 6. 모델 평가

In [184]:
# 라이브러리 가져오기
import pandas as pd

In [107]:
# 가스 데이터
가스 = pd.read_csv('./data/한국가스공사_월간 시도별 도시가스 판매현황_20201231.csv', encoding='cp949')

# 가스 데이터 개요
가스.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   연       384 non-null    int64
 1   월       384 non-null    int64
 2   강원      384 non-null    int64
 3   서울      384 non-null    int64
 4   경기      384 non-null    int64
 5   인천      384 non-null    int64
 6   경남      384 non-null    int64
 7   경북      384 non-null    int64
 8   광주      384 non-null    int64
 9   대구      384 non-null    int64
 10  대전      384 non-null    int64
 11  부산      384 non-null    int64
 12  세종      384 non-null    int64
 13  울산      384 non-null    int64
 14  전남      384 non-null    int64
 15  전북      384 non-null    int64
 16  제주      384 non-null    int64
 17  충남      384 non-null    int64
 18  충북      384 non-null    int64
dtypes: int64(19)
memory usage: 57.1 KB


In [108]:
가스.head()

,연,월,강원,서울,경기,인천,경남,경북,광주,대구,대전,부산,세종,울산,전남,전북,제주,충남,충북
0,1989,1,326,37560,0,10468,2942,150,2203,1940,322,3112,0,2006,764,586,0,0,0
1,1989,2,272,33566,0,9211,2940,139,2034,1756,345,3052,0,1900,722,864,0,0,0
2,1989,3,253,30224,0,9608,2702,271,1940,1896,268,3714,0,2135,734,980,0,0,0
3,1989,4,163,18668,0,7736,1951,239,1225,1416,257,2497,0,1402,533,600,0,0,0
4,1989,5,134,15790,0,7091,1564,359,903,1270,196,2268,0,1445,474,461,0,0,0


In [109]:
가스.tail()
# 가스 데이터는 1989년 1월 ~ 2020년 12월 존재

,연,월,강원,서울,경기,인천,경남,경북,광주,대구,대전,부산,세종,울산,전남,전북,제주,충남,충북
379,2020,8,19076,163032,255222,70646,50960,76285,36452,53419,32575,70508,93624,103814,40148,43885,1506,23090,7857
380,2020,9,18822,147556,243817,72542,53774,80552,37453,51142,30989,70849,100875,112923,44921,48190,1179,37694,7378
381,2020,10,25946,213556,339823,78654,62056,95788,50615,52524,43234,85283,120481,132347,49228,61328,1414,50694,8270
382,2020,11,35632,375289,497221,115663,84474,124036,71840,78450,71668,122943,165923,164927,55073,82360,2161,58945,10230
383,2020,12,57301,636178,742863,179536,124291,167410,115182,117002,114229,200163,232957,246272,77270,113880,4220,75782,13340


In [110]:
# 가스 데이터 전처리
가스 = 가스[['연', '월', '서울']]
가스.head()

,연,월,서울
0,1989,1,37560
1,1989,2,33566
2,1989,3,30224
3,1989,4,18668
4,1989,5,15790


In [188]:
# weather 데이터 1
weather1 = pd.read_csv('./data/2000-01~2021-11_월평균날씨.csv', encoding='cp949')

# 날씨 데이터 1 개요
weather1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   지점                   263 non-null    int64  
 1   지점명                  263 non-null    object 
 2   일시                   263 non-null    object 
 3   평균기온(°C)             263 non-null    float64
 4   평균최고기온(°C)           263 non-null    float64
 5   평균최저기온(°C)           263 non-null    float64
 6   최고기온(°C)             263 non-null    float64
 7   최저기온(°C)             263 non-null    float64
 8   최고기온 나타난날(yyyymmdd)  263 non-null    int64  
 9   최저기온 나타난날(yyyymmdd)  263 non-null    int64  
dtypes: float64(5), int64(3), object(2)
memory usage: 20.7+ KB


In [189]:
# weather1 1차 전처리
weather1['year'] = weather1.일시.str.split('-').str[0]
weather1['month'] = weather1.일시.str.split('-').str[1]
weather1

,지점,지점명,일시,평균기온(°C),평균최고기온(°C),평균최저기온(°C),최고기온(°C),최저기온(°C),최고기온 나타난날(yyyymmdd),최저기온 나타난날(yyyymmdd),year,month
0,108,서울,2000-01,-2.1,1.7,-5.7,9.9,-12.1,20000101,20000107,2000,01
1,108,서울,2000-02,-1.7,2.6,-5.8,7.5,-11.6,20000219,20000201,2000,02
2,108,서울,2000-03,6.3,11.4,1.7,18.9,-4.8,20000327,20000308,2000,03
3,108,서울,2000-04,11.9,16.9,7.3,23.5,1.8,20000420,20000411,2000,04
4,108,서울,2000-05,17.5,22.7,12.9,30.5,7.9,20000525,20000502,2000,05
...,...,...,...,...,...,...,...,...,...,...,...,...
258,108,서울,2021-07,28.1,32.2,24.6,36.5,19.6,20210724,20210703,2021,07
259,108,서울,2021-08,25.9,29.7,22.8,33.7,18.1,20210805,20210831,2021,08
260,108,서울,2021-09,22.6,26.9,18.8,30.3,16.0,20210913,20210930,2021,09
261,108,서울,2021-10,15.6,20.5,11.6,28.8,1.3,20211003,20211017,2021,10


In [190]:
# 월 데이터를 대표할 수 없는 최고기온 나타난날과 최저기온 날타난날 데이터를 제외
weather1 = weather1[['year','month','평균기온(°C)','평균최고기온(°C)','평균최저기온(°C)','최고기온(°C)','최저기온(°C)']]
weather1.astype({'year':'int', 'month':'int'})
weather1.head()

,year,month,평균기온(°C),평균최고기온(°C),평균최저기온(°C),최고기온(°C),최저기온(°C)
0,2000,01,-2.1,1.7,-5.7,9.9,-12.1
1,2000,02,-1.7,2.6,-5.8,7.5,-11.6
2,2000,03,6.3,11.4,1.7,18.9,-4.8
3,2000,04,11.9,16.9,7.3,23.5,1.8
4,2000,05,17.5,22.7,12.9,30.5,7.9


In [191]:
weather1.tail()
# 날씨1 데이터는 2000년 1월 ~ 2021년 11월 존재

,year,month,평균기온(°C),평균최고기온(°C),평균최저기온(°C),최고기온(°C),최저기온(°C)
258,2021,07,28.1,32.2,24.6,36.5,19.6
259,2021,08,25.9,29.7,22.8,33.7,18.1
260,2021,09,22.6,26.9,18.8,30.3,16.0
261,2021,10,15.6,20.5,11.6,28.8,1.3
262,2021,11,8.2,13.1,4.0,21.3,-3.7


In [166]:
# 날씨 데이터 2
날씨2 = pd.read_csv('./data/ta_20211230112152.csv', encoding='cp949', skiprows=7)

# 날씨 데이터 2 개요
날씨2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   년월       396 non-null    object 
 1   지점       396 non-null    int64  
 2   평균기온(℃)  396 non-null    float64
 3   최저기온(℃)  396 non-null    float64
 4   최고기온(℃)  396 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 15.6+ KB


In [167]:
# 날씨2 1차 전처리
날씨2['연'] = 날씨2.년월.str.split('-').str[0]
날씨2['월'] = 날씨2.년월.str.split('-').str[1]
날씨2

,년월,지점,평균기온(℃),최저기온(℃),최고기온(℃),연,월
0,1989-01,108,0.8,-11.2,11.2,1989,01
1,1989-02,108,2.4,-8.2,15.4,1989,02
2,1989-03,108,6.1,-5.0,16.1,1989,03
3,1989-04,108,14.3,3.7,29.4,1989,04
4,1989-05,108,17.9,8.4,29.6,1989,05
...,...,...,...,...,...,...,...
391,2021-08,108,25.9,18.1,33.7,2021,08
392,2021-09,108,22.6,16.0,30.3,2021,09
393,2021-10,108,15.6,1.3,28.8,2021,10
394,2021-11,108,8.2,-3.7,21.3,2021,11


In [168]:
날씨2 = 날씨2[['연','월','평균기온(℃)','최고기온(℃)','최저기온(℃)']]
날씨2.head()

,연,월,평균기온(℃),최고기온(℃),최저기온(℃)
0,1989,01,0.8,11.2,-11.2
1,1989,02,2.4,15.4,-8.2
2,1989,03,6.1,16.1,-5.0
3,1989,04,14.3,29.4,3.7
4,1989,05,17.9,29.6,8.4


In [169]:
날씨2.tail()
# 날씨2 데이터는 1989년 1월 ~ 2021년 12월 존재

,연,월,평균기온(℃),최고기온(℃),최저기온(℃)
391,2021,08,25.9,33.7,18.1
392,2021,09,22.6,30.3,16.0
393,2021,10,15.6,28.8,1.3
394,2021,11,8.2,21.3,-3.7
395,2021,12,1.0,13.3,-15.5


In [194]:
# 기계학습 1 : 날씨1 데이터를 이용한 날씨 예측
# 훈련 데이터 
trainX = weather1[weather1['year'] < '2020']
trainY = weather1[(weather1['year'] > '2000') & (weather1['year'] <= '2020')]

,year,month,평균기온(°C),평균최고기온(°C),평균최저기온(°C),최고기온(°C),최저기온(°C)
0,2000,01,-2.1,1.7,-5.7,9.9,-12.1
1,2000,02,-1.7,2.6,-5.8,7.5,-11.6
2,2000,03,6.3,11.4,1.7,18.9,-4.8
3,2000,04,11.9,16.9,7.3,23.5,1.8
4,2000,05,17.5,22.7,12.9,30.5,7.9
...,...,...,...,...,...,...,...
235,2019,08,27.2,31.6,23.6,36.8,18.5
236,2019,09,22.6,27.1,19.0,30.5,13.6
237,2019,10,16.4,21.8,11.8,28.2,4.7
238,2019,11,7.6,12.9,3.1,22.1,-4.6


In [ ]:
# 텐서플로우 임포트
import tesorflow as tf

In [ ]:
drop_rate=0.1

# 레이어 구성
# L1
X = tf.keras.layers.Input(shape=[입력노드갯수])
H = tf.keras.layers.BatchNarmalization(X)
H = tf.keras.layers.Activation('relu')(H)

# L2
H = tf.keras.layers.Dense(히든노드갯수)(H)
H = tf.keras.layers.BatchNarmalization(H)
H = tf.keras.layers.Activation('relu')(H)

# L3
H = tf.keras.layers.Dense(히든노드갯수)(H)
H = tf.keras.layers.BatchNarmalization(H)
H = tf.keras.layers.Activation('relu')(H)
H = tf.keras.layers.Dropout(drop_rate)

# L4
H = tf.keras.layers.Dense(히든노드갯수)(H)
H = tf.keras.layers.BatchNarmalization(H)
H = tf.keras.layers.Activation('relu')(H)

# L5
H = tf.keras.layers.Dense(출력노드갯수)(H)
H = tf.keras.layers.BatchNarmalization(H)
Y = tf.keras.layers.Activation('softmax')(H)

In [ ]:
# 모델 생성
model = tf.keras.models.Model(X,Y)